
# Оцінка та вдосконалення моделі


<h2>Мета</h2>

Ознайомитись з методами оцінки та вдосконалення регресійних моделей. Після завершення цієї лабораторної роботи ви зможете:

* Розділяти дані на навчальну та тестову вибірки
* Використовувати перехресну перевірку для оцінки якості моделі
* Обирати оптимальну складність моделі для уникнення перенавчання
* Вдосконалювати моделі прогнозування за допомогою підбору параметрів


<div class="alert alert-danger alertdanger" style="margin-top: 20px">

# <h1 id="task">Завдання, що оцінюються </h1>

</div>


1. Скачайте дані із файлу <a href="https://drive.google.com/file/d/1wSj5SKRPKOjNPP2cyqaNpxRCinKEmJkO/view?usp=drive_link" target="_blank">'clean_data2.csv'</a> (Data2.csv з виправленими помилками та заповненими пропусками). Запишіть дані у два датафрейми: предиктори (x_data) та відгуки (y_data). Випадковим чином розділіть дані на навчальні та тестові (використайте 20% загального набору в якості тестових даних).
2. Побудуйте модель лінійної одномірної регресії для одного з предикторів, використовуючи навчальні дані. Знайдіть R^2 на навчальних та тестових даних. Чому вони різні і на який з них треба орієнтуватись при виборі моделі?
3. Побудуйте кілька поліноміальних моделей різних ступенів для того ж предиктора. Знайдіть R^2 цих моделей на тестових даних. Яка з моделей краща? При якому ступені полінома спостерігається перенавчання?
4. Побудуйте модель гребеневої регресії для двох найкращих предикторів. Параметр alpha повинен бути встановлений на 10. Обчисліть R^2.
5. Виконайте пошук по сітці для параметра alpha та параметра нормалізації, а потім побудуйте модель, використавши найкращі значення параметрів.

<a name="task1"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task1">Завдання  #1: </h2>

<p>Запишіть дані у два датафрейми: предиктори (x_data) та відгуки (y_data). Випадковим чином розділіть дані на навчальні та тестові (використайте 20% загального набору в якості тестових даних).</p>
</div>


Зчитую дані з файлу у датафрейм


In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:

path = './clean_data2.csv'

In [17]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
df = pd.read_csv(path)
df

,Country Name,Region,GDP per capita,Population,CO2 emission,Area,Population density
0,Afghanistan,South Asia,561.778746,34656032.0,9809.225000,652860,53.083405
1,Albania,Europe & Central Asia,4124.982390,2876101.0,5716.853000,28750,100.038296
2,Algeria,Middle East & North Africa,3916.881571,40606052.0,145400.217000,2381740,17.048902
3,American Samoa,East Asia & Pacific,11834.745230,55599.0,165114.116337,200,277.995000
4,Andorra,Europe & Central Asia,36988.622030,77281.0,462.042000,470,164.427660
...,...,...,...,...,...,...,...
212,Virgin Islands (U.S.),Latin America & Caribbean,13445.593416,102951.0,165114.116337,350,294.145714
213,West Bank and Gaza,Middle East & North Africa,2943.404534,4551566.0,165114.116337,6020,756.074086
214,"Yemen, Rep.",Middle East & North Africa,990.334774,27584213.0,22698.730000,527970,52.245796
215,Zambia,Sub-Saharan Africa,1269.573537,16591390.0,4503.076000,752610,22.045136


Буду використовувати тільки числові дані


In [18]:

df=df._get_numeric_data()
df.head()

,GDP per capita,Population,CO2 emission,Area,Population density
0,561.778746,34656032.0,9809.225000,652860,53.083405
1,4124.982390,2876101.0,5716.853000,28750,100.038296
2,3916.881571,40606052.0,145400.217000,2381740,17.048902
3,11834.745230,55599.0,165114.116337,200,277.995000
4,36988.622030,77281.0,462.042000,470,164.427660


Записую дані у два датафрейми: предиктори (x_data) та відгуки (y_data).


In [19]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
y_data = df['CO2 emission']
x_data = df.drop('CO2 emission', axis=1)

Випадковим чином розділяю дані на навчальні та тестові (40% загального набору в якості тестових даних).


In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.40, random_state=0)


print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

number of test samples : 87
number of training samples: 130


<a name="task2"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task2">Завдання  #2: </h2>

<p>Побудуйте модель лінійної одномірної регресії для одного з предикторів, використовуючи навчальні дані. Знайдіть R^2 на навчальних та тестових даних. Чому вони різні і на який з них треба орієнтуватись при виборі моделі?</p>
</div>


<p>З попередніх робіт знаю, що хорошими показниками для 'CO2 emission' можуть бути:</p>
<ul>
    <li>ознака 1 -> Population</li>
    <li>ознака 2 -> Area</li>
</ul>
При цьому одномірна модель для ознаки 1 була кращою. Тому будуватиму моделі, використовуючи саме цю ознаку як предиктор.

<p>Створюю об’єкт лінійної регресії та навчаю модель</p>


In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
lre=LinearRegression()
lre.fit(x_train[['Population']], y_train)

LinearRegression()

Обчислюю R^2  на навчальних та тестових даних:


In [23]:
print("Testing data r^2: ", lre.score(x_test[['Population']], y_test))
print("Training data r^2: ", lre.score(x_train[['Population']], y_train))


Testing data r^2:  -0.5104728841567654
Training data r^2:  0.659413239052307


<p>Вони різні бо недостатньо даних  і відбулося перенавчання для тестових даних  </p>
<p>При виборі моделі треба орієнтуватись на її здатність до узагальнення(модель яка стабільна на всіх даних) на нові дані</p>


<a name="task3"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task3">Завдання  #3: </h2>

<p>Побудуйте кілька поліноміальних моделей різних ступенів для того ж предиктора. Знайдіть R^2 цих моделей на тестових даних. Яка з моделей краща? При якому ступені полінома спостерігається перенавчання?</p>
</div>


Будую поліноміальну модель 2 ступеня, на тестових даних обчислюю R^2


In [64]:
from sklearn.preprocessing import PolynomialFeatures

pr = PolynomialFeatures(degree=2)
# Виконую поліноміальне перетворення 2 ступеня для ознаки 'Population'
x_test_pr = pr.fit_transform(x_test[['Population']])
x_train_pr = pr.fit_transform(x_train[['Population']])

# Будую поліноміальну модель
lr = LinearRegression()
lr.fit(x_train_pr, y_train)

# На тестових даних обчислюю R^2
lr.score(x_test_pr, y_test)


-2.8021003390805106

Будую поліноміальну модель 3 ступеня, на тестових даних обчислюю R^2


In [66]:

pr3 = PolynomialFeatures(degree=3)

# Виконую поліноміальне перетворення 3 ступеня для ознаки 'Population'
x_test_pr3 = pr3.fit_transform(x_test[['Population']])
x_train_pr3 = pr3.fit_transform(x_train[['Population']])

# Будую поліноміальну модель
lr3 = LinearRegression()
lr3.fit(x_train_pr3, y_train)

# На тестових даних обчислюю R^2
lr3.score(x_test_pr3, y_test)

-0.043493939033268925

Будую поліноміальну модель 4 ступеня, на тестових даних обчислюю R^2


In [67]:
pr4 = PolynomialFeatures(degree=4)

# Виконую поліноміальне перетворення 3 ступеня для ознаки 'Population'
x_test_pr4 = pr4.fit_transform(x_test[['Population']])
x_train_pr4 = pr4.fit_transform(x_train[['Population']])

# Будую поліноміальну модель
lr4 = LinearRegression()
lr4.fit(x_train_pr4, y_train)

# На тестових даних обчислюю R^2
lr4.score(x_test_pr4, y_test)

-0.6231498854130431

Найкращою є модель 3 ступеня, бо для 4 ступеня вже спостерігається перенавчання.


<a name="task4"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task4">Завдання  #4: </h2>

<p>Побудуйте модель гребеневої регресії для двох найкращих предикторів. Параметр alpha повинен бути встановлений на 10. Обчисліть R^2.</p>
</div>


Будую модель гребеневої регресії, обчислюю R^2


In [68]:
# Виконую поліноміальне перетворення 2 ступеня для двох обраних ознак
pr=PolynomialFeatures(degree=2)
x_train_pr=pr.fit_transform(x_train[['Area', 'Population']])
x_test_pr=pr.fit_transform(x_test[['Area', 'Population']])

In [69]:
from sklearn.linear_model import Ridge
# Створюю об’єкт гребеневої регресії, встановивши параметр alpha=10
RigeModel=Ridge(alpha=10)

<a name="task5"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task5">Завдання  #5: </h2>

<p>Виконайте пошук по сітці для параметра alpha та параметра нормалізації, а потім побудуйте модель, використавши найкращі значення параметрів.</p>
</div>


Створюю словник значень параметрів:


In [71]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
parameters1= [{'alpha': [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 1000000]}]

Створюю об’єкт сітки пошуку параметра гребеневої регресії:


In [72]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
RR=Ridge()

Підбираю модель:


In [73]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
from sklearn.model_selection import GridSearchCV
Grid1 = GridSearchCV(RR, parameters1, cv=4)
Grid1.fit(x_data[['Area', 'Population']], y_data)

GridSearchCV(cv=4, estimator=Ridge(),
             param_grid=[{'alpha': [0.001, 0.1, 1, 10, 100, 1000, 10000, 100000,
                                    1000000]}])

Отримую модель з найкращими параметрами:


In [74]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання
BestRR=Grid1.best_estimator_
BestRR

Ridge(alpha=1000000)

Тестую модель на тестових даних (обчислюю R^2):


In [75]:
BestRR.score(x_test[['Area', 'Population']], y_test)


-1.0647044230643345

<a name="task6"></a>
<div class="alert alert-danger alertdanger" style="margin-top: 20px">

## <h2 id="task6"> Додаткове завдання: </h2>

<p>Побудуйте візуалізації для оцінки якості моделей (лінійної, кращої з поліноміальних та кращої з гребеневих)</p>
</div>


Використаю метод "predict", щоб спрогнозувати результати, а потім скористаюсь функцією "DistributionPlot", щоб відобразити розподіл прогнозованих результатів для тестових даних порівняно з фактичними для тестових даних.


In [ ]:
# Напишіть ваш код нижче та натисніть Shift+Enter для виконання


<h2>Виконав студент групи ІП-24<br>В'ячеслав Денисенко</h2>
